### Import required libraries

In [1]:
# Erath Engine API
import ee
# Show results/maps on the interactive map
import geemap

### Initialize the Earth Engine API

In [2]:
ee.Initialize()

### Load and define the region of interest (Tehran)

In [6]:
tehran_roi = (ee.FeatureCollection("FAO/GAUL/2015/level2")# Level1: Province , level2: County
              .filter(ee.Filter.eq("ADM2_NAME", "Tehran"))# Filter only Tehran City. It keep only the polygon where ADM2_NAME == "Tehran"
              .geometry() #  Convert the filtered FeatureCollection into a geometry (polygon) required by EE
              )

### Landsat-8 L2 images

In [ ]:
# ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6'] are [Blue, Green, Red, NIR, SWIR] bands of Landsat8
landsat8_dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
.filterBounds(tehran_roi)\
.filter(ee.Filter.lt('CLOUD_COVER', 20))\
.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6'])

# Compute NDBI for each image from landsat8_year
def add_ndbi(img):
    """
    Compute the Normalized Difference Built-up Index (NDBI) for a single Landsat 8 image.
    
    NDBI highlights built-up areas using the SWIR and NIR bands:
        NDBI = (SWIR - NIR) / (SWIR + NIR)
    
    Args:
        img (ee.Image): A single Landsat 8 Surface Reflectance image.
        
    Returns:
        ee.Image: The input image with an additional band named "NDBI".
    """
    NIR = img.select("SR_B5")
    SWIR = img.select("SR_B6")
    NDBI = SWIR.subtract(NIR).divide(SWIR.add(NIR)).rename("NDBI")
    return img.addBands(NDBI)

def get_rgb(year):
    """
    Create a median RGB composite for a given year for visual inspection.
    
    Args:
        year (int): The year for which to create the RGB composite.
        
    Returns:
        ee.Image: Median composite image of the given year, clipped to Tehran ROI,
                  containing only the Red, Green, and Blue bands (SR_B4, SR_B3, SR_B2).
    """
    start_date = f'{year}-05-01'
    end_date = f'{year}-09-30'
    
    landsat8_year = landsat8_dataset.filterDate(start_date, end_date)
    
    # Take median composite and clip to ROI
    rgb_composite = landsat8_year.median().clip(tehran_roi)
    
    return rgb_composite.select(['SR_B4', 'SR_B3', 'SR_B2'])  # R, G, B

def get_annual_ndbi(year):
    """
    Compute the annual NDBI median composite for a given year and region of interest.
    
    Args:
        year (int): The year for which to compute NDBI.
        roi (ee.Geometry): The region of interest to clip the final composite.
        
    Returns:
        ee.Image: Median NDBI composite for the year, clipped to the ROI.
    """
    # Define seasonal window (May–September)
    start_date = f'{year}-05-01'
    end_date = f'{year}-09-30'
    
    # Filter the Landsat 8 collection for this year
    landsat8_year = landsat8_dataset.filterDate(start_date, end_date)
    
    # Compute NDBI for each image
    landsat8_year_ndbi = landsat8_year.map(add_ndbi) # all original bands of Landsat 8 (SR_B2–SR_B6) plus the added NDBI band
    
    # Take the median and clip to ROI
    composite = landsat8_year_ndbi.select('NDBI').median().clip(tehran_roi) # Only NDBI band
    
    return composite

# define taks to downlaod the images in your Google Drive
for year in range(2013, 2024):
    ndbi_img = get_annual_ndbi(year)
    task = ee.batch.Export.image.toDrive(
        image= ndbi_img,
        description = f'NDBI_{year}_Tehran',
        folder = 'Tehran_NDBI', # mannualy create a folder in your Gogole Drive called Tehran_NDBI, otherwise command out this
        fileNamePrefix= f'NDBI_{year}',
        region = tehran_roi,
        scale = 30,
        maxPixels= 1e10,
        fileFormat='GeoTIFF',
        crs= 'EPSG:4326'
    )
    task.start()
    print(f"Export started for {year}")
